In [1]:
import pandas as pd
import numpy as np

In [20]:
listpages = pd.read_parquet('./fct_list_pages.parque')
listpages['containerId'] = listpages['eventId'].map(lambda e: e.split('.')[0])
pageview = pd.read_parquet('./fct_page_view_nl.parque')
slots = pd.read_parquet('./export-3-nov/fct_slot_interaction/')

In [21]:
slots = slots.convert_dtypes()
slots['eventTimestamp'] = pd.to_datetime(slots['eventTimestamp'])
slots.dtypes

visitId           string[python]
eventTimestamp    datetime64[us]
groupId           string[python]
pageName          string[python]
groupType         string[python]
groupName         string[python]
groupTitle        string[python]
hasView                  boolean
hasClick                 boolean
isWebsite                  Int64
isAndroid                  Int64
isIOS                      Int64
dtype: object

In [9]:
pageview['containerId'] = pageview['eventId']
pageview_min = pageview[['containerId', 'pageName']]
listpages_named = pd.merge_ordered(how='inner', left=listpages, on='containerId', right=pageview_min)
listpages_named.shape

(212032, 14)

In [23]:
np.unique(slots['pageName'].tolist())

array(['ACCOUNT', 'ADDONPAGE', 'BRAND', 'CUSTOMERSERVICE', 'EXPLORE',
       'HOME', 'OHP', 'PDP', 'RA', 'SEARCH', 'UNKNOWN'], dtype='<U15')

In [24]:
splits = pageview['hookId'].str.extract(r'^(.*?)\.(.*)\.(.*?)$')

In [26]:
pageview['eventId'] = splits[0]
pageview['itemId'] = splits[1]
pageview['callToAction'] = splits[2]

In [51]:
added_cart = pageview.loc[pageview['callToAction'] == 'AddToCart']
payment = pageview.loc[pageview['pageName'] == 'PaymentDone']

visit_id = np.unique(added_cart['visitId'].to_list())
visit_id2 = np.unique(payment['visitId'].to_list())

len(visit_id2) *100 / len(visit_id)

83.08148566586641

In [55]:
#all visitIDs where an item is put into cart and bought in the same visitID
bought_session = list(set(visit_id2).intersection(visit_id))

In [70]:
test = bought_session[777]

tmp = slots.loc[slots['visitId'] == test]
tmp = tmp.sort_values('eventTimestamp', ascending=True)
tmp2 = added_cart.loc[added_cart['visitId'] == test]

In [77]:
concatenated_df = pd.concat([tmp, tmp2], axis=0)

concatenated_df

,visitId,eventTimestamp,groupId,pageName,groupType,groupName,groupTitle,hasView,hasClick,isWebsite,isAndroid,isIOS,_surrogate_key,dateAmsterdam,eventId,hookId,itemId,callToAction
334741,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 13:50:47.080,hC9qLTe0Gmoy5m3L0pZF7Q.5_10,HOME,Flexbanner,Flexbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
374346,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 13:50:47.080,hC9qLTe0Gmoy5m3L0pZF7Q.68_69,HOME,DayDeal,List,Dagdeal,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
7323980,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 13:50:47.080,hC9qLTe0Gmoy5m3L0pZF7Q.68_71,HOME,Flexbanner,Flexbanner,<NA>,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1082195,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 13:50:47.080,hC9qLTe0Gmoy5m3L0pZF7Q.2_3,HOME,Topbanner,Topbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1083535,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 13:50:47.080,hC9qLTe0Gmoy5m3L0pZF7Q.5_7,HOME,Flexbanner,Flexbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5007324,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 13:50:47.080,hC9qLTe0Gmoy5m3L0pZF7Q.5_12,HOME,Flexbanner,Flexbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1644454,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 13:50:47.080,hC9qLTe0Gmoy5m3L0pZF7Q.5_14,HOME,Flexbanner,Flexbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
427,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 14:05:45.625,j4vCrnJ1YEoyOnzQX3JFIQ.71_72,HOME,DayDeal,List,Dagdeal,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8071481,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 14:05:45.625,j4vCrnJ1YEoyOnzQX3JFIQ.5_12,HOME,Flexbanner,Flexbanner,<NA>,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
7510839,y5Jtd8oZzloly7Vfeoju2VIEvZC1PA2r0jOGnD+Z+EU,2022-10-31 14:05:45.625,j4vCrnJ1YEoyOnzQX3JFIQ.71_74,HOME,Flexbanner,Flexbanner,<NA>,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
